In [10]:
%reload_ext autoreload
%autoreload 2

from func import *
import pandas as pd



In [11]:
filename = 'all_data_anonymized.csv'
date_col = 'todate'
start_date = '1980-01-01'
seperator = ';'
fill = 0
lim = 5
df_read = load_and_preprocess_data(filename, date_col, start_date, seperator, fill, lim)


c:\Users\oscar.azrak\Desktop\Master-thesis\Master-thesis\Files\func.py:21: DtypeWarning: Columns (15,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df_read = pd.read_csv(filename, sep=seperator)


In [12]:
windows = [5, 10, 20, 40, 60, 100, 180, 240, 360, 480]
window_m = [10, 30, 60, 100, 180]
windows_test = [9]
assets = df_read.columns
df_feat = add_features(df_read, window_m)

In [13]:
target_days = 9

return_column_shift = 'avgreturn'
volatility_column_shift = 'volatility'
df = add_y_col(df_feat, df_read, date_col, target_days, return_column_shift, volatility_column_shift)

In [14]:
X_train, X_eval, X_test, y_train, y_eval, y_test, X_train_eval, y_train_eval = prepare_training_dataset(df, date_col, shuffle=False, train_split=0.25, eval_split=0.25)

In [15]:
# check if df contains nan

print(df.isnull().sum().sum())
print(df.head())

0
           todate   asset    VaR_10   VaR_100   VaR_180    VaR_30    VaR_60  \
120284 1997-07-25   EQS_0 -0.012419 -0.027867 -0.021799 -0.017527 -0.018084   
120285 1997-07-25   EQS_1 -0.017053 -0.020400 -0.020233 -0.019890 -0.018676   
120286 1997-07-25  EQS_10 -0.021068 -0.022818 -0.018827 -0.022964 -0.022541   
120287 1997-07-25   EQS_2 -0.012701 -0.014182 -0.011899 -0.016148 -0.012140   
120288 1997-07-25   EQS_3 -0.015363 -0.015846 -0.012966 -0.015688 -0.012114   

        avgreturn_10  avgreturn_100  avgreturn_180  avgreturn_30  \
120284      0.002417       0.000965       0.001501      0.001368   
120285     -0.000581       0.001318       0.001369      0.000985   
120286     -0.000989       0.000404       0.000697     -0.001385   
120287      0.001843       0.001058       0.000796      0.001652   
120288      0.001626       0.001758       0.001219      0.001664   

        avgreturn_60  momentum_10  momentum_100  momentum_180  momentum_30  \
120284      0.002340     0.024167   

In [16]:
param_grid_alpha = {'alpha': [0.1, 1.0, 10.0]}
ridge_best, grid_search = optimize_and_train_ridge(X_train, y_train, X_train_eval, y_train_eval, param_grid_alpha)

Best parameters: {'alpha': 10.0}
Best accuracy on evaluation set: 0.5086063469241039


In [17]:
param_grid_xgb = {
    'max_depth': [3, 6, 10],
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2]
}

xgb_best, best_params = optimize_and_train_xgb(X_train, y_train, X_eval, y_eval, param_grid_xgb)


C:\Users\oscar.azrak\AppData\Roaming\Python\Python310\site-packages\xgboost\sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


Best hyperparameters: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 50}


In [18]:

param_grid_lgb = {
    'max_depth': [3, 6, 10],
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'num_leaves': [31, 63, 127, 255]
}

# Call the function with your datasets and hyperparameter grid
lgb_best, best_params = optimize_and_train_lgb(X_train, y_train, X_eval, y_eval, param_grid_lgb)

In [ ]:
NN_model, history, X_test_scaled = train_and_evaluate_NN(X_train_eval, y_train_eval, X_eval, y_eval, X_test, y_test)


In [ ]:

# Store predictions in a dictionary
predictions = {
    "Ridge Classifier": ridge_best.predict(X_test),
    "XGBoost Classifier": xgb_best.predict(X_test),
    "LightGBM Classifier": lgb_best.predict(X_test),
}   

# Iterate through the dictionary and evaluate each model
for model_name, y_pred in predictions.items():
    print(model_name + ":")
    evaluate_model_performance(y_test, y_pred)
    print("\n")
